In [1]:
# load packages:
# package requirement: tensorflow-gpu 1.15.2, keras 2.3.1 munkres
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as pathces
import sys

from mmDUFS import JointModel
from mmDUFS import DiffModel
from mmDUFS import DataSet

from baselines import *

from sklearn.metrics import f1_score
from scipy import stats

Using TensorFlow backend.


In [2]:
!nvidia-smi

Thu Jun  1 01:39:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 29%   31C    P8    15W / 250W |  10775MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 31%   

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [4]:
# load the data
toy_data = np.load("./datasets/rotating_toy.npz",allow_pickle=True)
X_z=toy_data['X']
Y_z=toy_data['Y']
gt_feat_joint_X=toy_data['gt_feat_joint_X']
gt_feat_joint_Y=toy_data['gt_feat_joint_Y']
gt_feat_diff_X=toy_data['gt_feat_diff_X']
gt_feat_diff_Y=toy_data['gt_feat_diff_Y']

In [5]:
# run baselines
baseline_f1 = fs_eval(X = X_z,
        Y =  Y_z,
        label_true_X = gt_feat_joint_X,
        label_true_Y = gt_feat_joint_Y,
        baselines = ["concat","sum","prod"],
        nx=np.sum(gt_feat_joint_X == 1),
        ny=np.sum(gt_feat_joint_Y == 1), 
        knn=2,fac=5, laplacian="normalized")
   

In [6]:
baseline_f1

{'concat': {'X': 0.23665893271461716, 'Y': 0.3850574712643678},
 'sum': {'X': 0.5498839907192575, 'Y': 0.6925287356321839},
 'prod': {'X': 0.5498839907192575, 'Y': 0.6925287356321839}}

In [ ]:
# run mmdufs with shared op

In [7]:
shared_mmdufs_params = {
    "lam1":2e-1, # lambda x
    "lam2":2e-1, # lambda y
    "batch_size":X_z.shape[0], # full batch
    "const2":1e3
}
shared_mmdfus_learning_rate = 2
display_step = 1000
epochs = 20000

In [8]:
dataset = DataSet(**{'_data1':X_z,
                         '_data2':Y_z}
                         )

shared_mmdufs_params["input_dim1"] = X_z.shape[1]
shared_mmdufs_params["input_dim2"] = Y_z.shape[1]
    
model = JointModel(**shared_mmdufs_params) 
    
shared_result= model.train(dataset,
                            learning_rate=shared_mmdfus_learning_rate,
                        feature_label1 = gt_feat_joint_X, 
                        feature_label2 = gt_feat_joint_Y,
                        display_step=display_step, 
                        num_epoch=epochs) 







The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



num_samples : 4050
Epoch: 1000 loss= 0.195578963 score1= -0.066082649 score2= -0.067704506 reg1= 0.823448539 reg2= 0.823382080 f1 - Mod1 =  0.2471 f1 - Mod2 =  0.1674
Epoch: 2000 loss= 0.192074656 score1= -0.064861238 score2= -0.064921051 reg1= 0.804769874 reg2= 0.804514766 f1 - Mod1 =  0.2174 f1 - Mod2 =  0.1425
Epoch: 3000 loss= 0.185475126 score1= -0.064416528 score2= -0.064154439 reg1= 0.785388768 reg2= 0.784841716 f1 - Mod1 =  0.2178 f1 - Mod2 =  0.1364
Epoch: 4000 loss= 0.181906357 score1= -0.061202880 score2= -0.062853

In [16]:
f1_score(gt_feat_joint_X,model.get_prob_alpha1() == 1)

0.8252427184466019

In [17]:
f1_score(gt_feat_joint_Y,model.get_prob_alpha2() == 1)

0.7647831800262812

In [ ]:
# run mmdufs with diff op

In [18]:
diff_mmdufs_params = {
    "lam1":2, # lambda x
    "batch_size":X_z.shape[0], # full batch
    "const":3,
    "const2":1e3
}
diff_mmdfus_learning_rate = 2
display_step = 1000
epochs = 10000

In [19]:
diff_mmdufs_params["input_dim1"] = X_z.shape[1]
diff_mmdufs_params["input_dim2"] = Y_z.shape[1]

In [20]:
dataset = DataSet(**{'_data1':X_z,
                         '_data2':Y_z,
                    })
    
diff_model1 = DiffModel(**diff_mmdufs_params) 
    
diff_result1= diff_model1.train(dataset,
                              learning_rate=diff_mmdfus_learning_rate,
                        feature_label = gt_feat_diff_X.reshape(-1), 
                        display_step=display_step, 
                        num_epoch=epochs) #30000
  

num_samples : 4050
Epoch: 1000 loss= -12.984756470 score1= -13.390271187 reg1= 0.202757224 f1= 0.8727
Epoch: 2000 loss= -13.451340675 score1= -13.841081619 reg1= 0.194870621 f1= 0.8763
Epoch: 3000 loss= -13.634535789 score1= -14.021030426 reg1= 0.193247125 f1= 0.8789
Epoch: 4000 loss= -13.645084381 score1= -14.031055450 reg1= 0.192985579 f1= 0.8789
Epoch: 5000 loss= -13.654281616 score1= -14.039600372 reg1= 0.192659169 f1= 0.8794
Epoch: 6000 loss= -13.660076141 score1= -14.045257568 reg1= 0.192590684 f1= 0.8794
Epoch: 7000 loss= -13.658875465 score1= -14.043967247 reg1= 0.192546055 f1= 0.8794
Epoch: 8000 loss= -13.696744919 score1= -14.081356049 reg1= 0.192305684 f1= 0.8800
Epoch: 9000 loss= -13.697546005 score1= -14.082098961 reg1= 0.192276374 f1= 0.8800
Epoch: 10000 loss= -13.696771622 score1= -14.081283569 reg1= 0.192255899 f1= 0.8800
Optimization Finished!


In [21]:
f1_score(gt_feat_diff_X,1*(diff_model1.get_prob_alpha1() == 1))

0.8799526907155529

In [22]:
diff_mmdufs_params = {
    "lam1":2, # lambda x
    "batch_size":X_z.shape[0], # full batch
    "const":3,
    "const2":1e3
}
diff_mmdfus_learning_rate = 2
display_step = 1000
epochs = 10000

In [23]:
diff_mmdufs_params["input_dim1"] = Y_z.shape[1]
diff_mmdufs_params["input_dim2"] = X_z.shape[1]

In [24]:
dataset = DataSet(**{'_data1':Y_z,
                         '_data2':X_z,
                    })
    
diff_model2 = DiffModel(**diff_mmdufs_params) 
    
diff_result2 = diff_model2.train(dataset,
                              learning_rate=diff_mmdfus_learning_rate,
                        feature_label = gt_feat_diff_Y.reshape(-1), 
                        display_step=display_step, 
                        num_epoch=epochs) #30000
  

num_samples : 4050
Epoch: 1000 loss= 0.330438554 score1= -0.648107588 reg1= 0.489273071 f1= 0.6575
Epoch: 2000 loss= -12.149414062 score1= -12.469688416 reg1= 0.160136968 f1= 0.7415
Epoch: 3000 loss= -12.188891411 score1= -12.507352829 reg1= 0.159230575 f1= 0.7415
Epoch: 4000 loss= -12.212779045 score1= -12.530331612 reg1= 0.158776402 f1= 0.7419
Epoch: 5000 loss= -12.221035957 score1= -12.538360596 reg1= 0.158662468 f1= 0.7419
Epoch: 6000 loss= -12.297073364 score1= -12.613844872 reg1= 0.158385903 f1= 0.7424
Epoch: 7000 loss= -12.299599648 score1= -12.616278648 reg1= 0.158339724 f1= 0.7424
Epoch: 8000 loss= -12.300900459 score1= -12.617516518 reg1= 0.158307850 f1= 0.7424
Epoch: 9000 loss= -12.344826698 score1= -12.660978317 reg1= 0.158075601 f1= 0.7429
Epoch: 10000 loss= -12.347266197 score1= -12.663382530 reg1= 0.158058167 f1= 0.7429
Optimization Finished!


In [25]:
f1_score(gt_feat_diff_Y,1*(diff_model2.get_prob_alpha1() == 1))

0.7428571428571428